In [2]:
import numpy as np
import pandas as pd
import dill
from src.utils import *
from src.augmentator import Augmentator
from src.image_sequence_generator import ImageSequenceGenerator
from src.video_classifier import VideoClassifier
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from matplotlib import pyplot as plt

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [3]:
train_ids, train_classes = read_ids('../IDs/train.csv')
valid_ids, valid_classes = read_ids('../IDs/valid.csv')
test_ids, test_classes = read_ids('../IDs/test.csv')

In [3]:
video_path = '../videos'
train_augmentator = Augmentator(preprocessing_function=scale_image)
valid_augmentator = Augmentator(preprocessing_function=scale_image)
train_generator = ImageSequenceGenerator(
    video_path,
    train_ids,
    train_classes,
    ['0', '1'],
    train_augmentator,
    fit_eval=True,
    fps=3,
    sequence_time=2,
    batch_size=3
)
valid_generator = ImageSequenceGenerator(
    video_path,
    valid_ids,
    valid_classes,
    ['0', '1'],
    valid_augmentator,
    fit_eval=True,
    fps=3,
    sequence_time=2,
    batch_size=3
)

In [4]:
%%time
video_classifier = VideoClassifier(input_shape=(train_generator.timesteps, 224, 224, 3))
video_classifier.build_model()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Wall time: 25.7 s


In [5]:
%%time
video_classifier.fit(train_generator, epochs=1000, validation_data=valid_generator)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
 - 2853s - loss: 0.4641 - val_loss: 0.9528

Epoch 00001: val_loss improved from inf to 0.95278, saving model to model/checkpoint_best_model.h5
Epoch 2/1000
 - 2607s - loss: 0.4042 - val_loss: 0.5444

Epoch 00002: val_loss improved from 0.95278 to 0.54437, saving model to model/checkpoint_best_model.h5
Epoch 3/1000
 - 2582s - loss: 0.3224 - val_loss: 1.3720

Epoch 00003: val_loss did not improve from 0.54437
Epoch 4/1000
 - 2588s - loss: 0.3039 - val_loss: 0.4451

Epoch 00004: val_loss improved from 0.54437 to 0.44508, saving model to model/checkpoint_best_model.h5
Epoch 5/1000
 - 2586s - loss: 0.2724 - val_loss: 0.3800

Epoch 00005: val_loss improved from 0.44508 to 0.38001, saving model to model/checkpoint_best_model.h5
Epoch 6/1000
 - 2576s - loss: 0.2491 - val_loss: 0.4222

Epoch 00006: val_loss did not improve from 0.38001
Epoch 7/1000
 - 2584s - loss: 0.2297 - val_loss: 0.4930

Epoch 00007: val_loss did not improve from 

In [7]:
with open('model/history.pkl', 'wb') as f:
    dill.dump(video_classifier.history, f)

In [4]:
%%time
video_classifier = VideoClassifier(input_shape=(train_generator.timesteps, 224, 224, 3))
video_classifier.build_model(weights_path='model/checkpoint_best_model.h5')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Wall time: 28 s


In [5]:
%%time
with open('model/history.pkl', 'rb') as f:
    history = dill.load(f)

Instructions for updating:
Use tf.cast instead.
Wall time: 55.8 s


In [26]:
plot_learning_curves(history)

In [7]:
predict_augmentator = Augmentator(
    rotation_range=(0, 0),
    width_shift_range=(0, 0),
    height_shift_range=(0, 0),
    scale_x_range=(1, 1),
    scale_y_range=(1, 1),
    horizontal_flip=False,
    preprocessing_function=scale_image
)

In [9]:
train_predict_generator = ImageSequenceGenerator(
    video_path,
    train_ids,
    train_classes,
    ['0', '1'],
    predict_augmentator,
    fps=3,
    sequence_time=2,
    batch_size=3,
    shuffle=False
)
valid_predict_generator = ImageSequenceGenerator(
    video_path,
    valid_ids,
    valid_classes,
    ['0', '1'],
    predict_augmentator,
    fps=3,
    sequence_time=2,
    batch_size=3,
    shuffle=False
)
test_predict_generator = ImageSequenceGenerator(
    video_path,
    test_ids,
    test_classes,
    ['0', '1'],
    predict_augmentator,
    fps=3,
    sequence_time=2,
    batch_size=3,
    shuffle=False
)

In [10]:
%%time
y_true_train, y_pred_train = video_classifier.predict(train_predict_generator)

Wall time: 20min 33s


In [11]:
%%time
y_true_valid, y_pred_valid = video_classifier.predict(valid_predict_generator)

Wall time: 18min 54s


In [12]:
%%time
y_true_test, y_pred_test = video_classifier.predict(test_predict_generator)

Wall time: 17min 27s


In [27]:
fpr_train, tpr_train, roc_thresholds_train = roc_curve(y_true_train[:, 1], y_pred_train[:, 1])
fpr_valid, tpr_valid, roc_thresholds_valid = roc_curve(y_true_valid[:, 1], y_pred_valid[:, 1])
fpr_test, tpr_test, roc_thresholds_test = roc_curve(y_true_test[:, 1], y_pred_test[:, 1])
plot_curves(
    fpr_train,
    tpr_train,
    roc_thresholds_train,
    fpr_valid,
    tpr_valid,
    roc_thresholds_valid,
    fpr_test,
    tpr_test,
    roc_thresholds_test,
    output='plot/ROC_curves.html'
)

In [28]:
recall_train, precision_train, pr_thresholds_train = precision_recall_curve(y_true_train[:, 1], y_pred_train[:, 1])
recall_valid, precision_valid, pr_thresholds_valid = precision_recall_curve(y_true_valid[:, 1], y_pred_valid[:, 1])
recall_test, precision_test, pr_thresholds_test = precision_recall_curve(y_true_test[:, 1], y_pred_test[:, 1])
plot_curves(
    recall_train,
    precision_train,
    pr_thresholds_train,
    recall_valid,
    precision_valid,
    pr_thresholds_valid,
    recall_test,
    precision_test,
    pr_thresholds_test,
    curve_type='PR',
    output='plot/PR_curves.html'
)

In [19]:
roc_auc = roc_auc_score(y_true_train[:, 1], y_pred_train[:, 1])
gini = gini_score(roc_auc)
display('ROC AUC and GINI train:', roc_auc, gini)

'ROC AUC and GINI train:'

0.9762501722926521

0.9525003445853042

In [21]:
roc_auc = roc_auc_score(y_true_valid[:, 1], y_pred_valid[:, 1])
gini = gini_score(roc_auc)
display('ROC AUC and GINI valid:', roc_auc, gini)

'ROC AUC and GINI valid:'

0.9448443634887268

0.8896887269774536

In [22]:
roc_auc = roc_auc_score(y_true_test[:, 1], y_pred_test[:, 1])
gini = gini_score(roc_auc)
display('ROC AUC and GINI test:', roc_auc, gini)

'ROC AUC and GINI test:'

0.8252614845295787

0.6505229690591574

In [23]:
print('PR Train:', auc(recall_train, precision_train, reorder=True))
print('PR Valid:', auc(recall_valid, precision_valid, reorder=True))
print('PR Test:', auc(recall_test, precision_test, reorder=True))

PR Train: 0.3723167918288589
PR Valid: 0.41541187105997757
PR Test: 0.3434730382750549


In [10]:
mark_video_path = '../videos_old_raw/raw/test/0'
mark_video_generator = ImageSequenceGenerator(
    mark_video_path,
    ids=['Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4'],
    fps=3,
    sequence_time=2,
    batch_size=3,
    shuffle=False,
    fit_eval=False
)

In [15]:
mark_video_generator.start_positions

[('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  0),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  27),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  54),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  81),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  108),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  135),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  162),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  189),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  216),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4',
  243),
 ('Full Match - Arsenal FC vs. FC Bayern 2-1 - International Cham

In [19]:
mark_video_generator.shift_time

1

In [16]:
mark_video_generator.shift_frames

3

In [8]:
video_classifier = VideoClassifier(input_shape=(mark_video_generator.timesteps, 224, 224, 3))
video_classifier.build_model(weights_path='model/checkpoint_best_model.h5')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
%%time
y_pred_mark = video_classifier.predict(mark_video_generator)

Wall time: 39min 35s


In [18]:
trace = go.Scatter(
    x=np.arange(0, y_pred_mark.shape[0]),
    y=y_pred_mark[:, 1]
)
data = [trace]
layout = go.Layout(
    title=dict(
        text='Violation distribution'
    ),
    xaxis=dict(
        title=dict(
            text='Second'
        )
    ),
    yaxis=dict(
        title=dict(
            text='Score per second'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='plot/violation_distribution.html')

'plot/violation_distribution.html'

In [46]:
def hms(sec):
    seconds = sec
    hours = int(seconds / 3600)
    seconds = int(seconds - hours * 3600)
    minutes = int(seconds / 60)
    seconds = int(seconds - minutes * 60)
    return hours, minutes, seconds

In [61]:
trace = go.Scatter(
    x=list(map(lambda x: '{:02}:{:02}:{:02}'.format(x[0], x[1], x[2]), map(hms, range(y_pred_mark.shape[0])))),
    y=y_pred_mark[:, 1]
)
data = [trace]
layout = go.Layout(
    title=dict(
        text='Violation distribution'
    ),
    xaxis=dict(
        title=dict(
            text='Second'
        )
    ),
    yaxis=dict(
        title=dict(
            text='Score per second'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='plot/violation_distribution.html')

'plot/violation_distribution.html'

In [9]:
video_classifier.timesteps

AttributeError: 'VideoClassifier' object has no attribute 'timesteps'